In [ ]:
using LinearAlgebra

Let's create a custom nxn matrix that depends on n paramaters, not $n^2$: <br>

M = diag(v) + v*v'

Here M.v will denote the parameters that M depends on. <br>
This is not a sparse matrix, but dense linear algebra clearly does not apply.  This is way more important than sparse in applications.

In [ ]:
## This would work, but we'll add a few fancy features
## struct Custom
##     v::Vector
## end

In [ ]:
struct Custom{T} <: AbstractMatrix{T}
     v::Vector{T}
end

We added the type parameter {T} and the subtype AbstractMatrix.  The type parameters says that the vector could be made from any type: Int, Float, BigInt, Quaternion,... <br>

The subtype of AbstractMatrix automatically gives some functionality of size and getindex methods are defined as in the next cell.

In [6]:
Base.size(c::Custom)         =   length(c.v), length(c.v)
Base.getindex(c::Custom,i,j) =   c.v[i]*(i==j)  + c.v[i]*c.v[j]

In [7]:
M = Custom([1, 2, 3])

3×3 Custom{Int64}:
 2  2   3
 2  6   6
 3  6  12

In [10]:
Matrix(M)

3×3 Array{Int64,2}:
 2  2   3
 2  6   6
 3  6  12

Notice the different types.  M would generally store n numbers, Matrix(M) would store $n^2$ numbers.

In [12]:
M * [1,2,3] # Matrix times vector automatically works though may not be efficient.

3-element Array{Int64,1}:
 15
 32
 51

In [15]:
# Matrix times vector with O(n) operations (3n multiplications , 2n additions)
Base.:*(c::Custom, x::Vector) = (c.v) .* x + c.v * (c.v'x)

In [17]:
M * [1,2,3] # same answer, now efficient

3-element Array{Int64,1}:
 15
 32
 51

In [ ]:
#using Pkg
#Pkg.add("Arpack")
#using Arpack
svds(c,nsv=4)

In [ ]:


#matrix(c::Custom) = Diagonal(c.v) + c.v*c.v'

f(c::Custom)  = λ ->  1 + sum(c.v[i]^2 / (c.v[i] - λ)   for i=1:length(c.v))
f′(c::Custom) = λ ->      sum(c.v[i]^2 / (c.v[i] - λ)^2 for i=1:length(c.v))

In [ ]:
v = randn(3)
c = Custom(v)

In [ ]:
# Newtons method
function LinearAlgebra.eigmax(c::Custom; tol = eps(2.0), debug = false)
    x0 = maximum(c.v) + maximum(c.v)^2
    δ = f(c)(x0)/f′(c)(x0)
    while abs(δ) > x0 * tol               
        x0 -= δ
        δ = f(c)(x0)/f′(c)(x0)
        debug && println("x = $x0, δ = $δ")
    end
    x0
end

In [ ]:
#v = randn(3000)
c = Custom(v)
m = Symmetric(Matrix(c))
eigmax(m)

In [ ]:
@time maximum(eigvals(m))

In [ ]:
@time eigmax(m)

In [ ]:
eigmax(c,debug=true)

In [ ]:
@time eigmax(c)

In [ ]:
using Plots
pyplot()

In [ ]:
n=5
v = randn(n)
c = Custom(v)

xmin = maximum(v)
xmax = 2*maximum(eigvals(matrix(c))) - xmin

plot(f(c),xmin,xmax, ylim=(-3,1))
plot!(x->0,xmin,xmax,color=:red)
plot!(x->1,xmin,xmax, color=:red)
scatter!([maximum(eigvals(matrix(c)))],[0.0])

x = xmin + xmin^2 
scatter!([x],[f(c)(x)])
for k=2:5
    x -= f(c)(x)/f′(c)(x) 
    scatter!([x],[f(c)(x)])
end
plot!()

In [ ]:
?eigmax

In [ ]:
Base.:*(c::Custom, x::Vector) = (c.v) .* x + c.v * (c.v'x)

In [ ]:
c * randn(5)

In [ ]:
#using Pkg
#Pkg.add("Arpack")
#using Arpack
svds(c,nsv=4)